In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
import re
from cleantext import clean
from googletrans import Translator
translator = Translator()

from os.path import exists

import configparser
import tweepy
import pandas as pd
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to C:\Users\SAMEER
[nltk_data]     MISHRA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SAMEER
[nltk_data]     MISHRA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def clean_text(each_text):
    #Translating text to english
    translated_text = translator.translate(each_text,dest="en")
    
    #removing emoji
    each_text_no_emoji = clean(translated_text.text, no_emoji=True)
    
    #removing URL from text
    each_text_no_url = re.sub(r"http\S+", "", each_text_no_emoji)
        
    # removing numbers from text
    text_no_num = re.sub(r'\d+', '', each_text_no_url)

    # tokenizing each text
    word_tokens = word_tokenize(text_no_num)
    
    # removing special character
    clean_text = []
    for word in word_tokens:
        clean_text.append("".join([z for z in word if z.isalnum()]))

    # removing stop words and lower
    text_with_no_stop_word = [w.lower() for w in clean_text if not w in stop_words]  

    # do stemming
    stemmed_text = [stemmer.stem(w) for w in text_with_no_stop_word]
    
    return " ".join(" ".join(stemmed_text).split())

In [11]:
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

column = ['Time','User','Tweet']

if exists('Dataset/tweets.csv'):
    data=[]
else:
    data = [column]

officialHandles = ['NDRFHQ','ndmaindia','DisasterState']
for i in officialHandles:
    public_tweets = api.user_timeline(screen_name=i, 
                               count=200,
                               include_rts = False,
                               tweet_mode = 'extended'
                               )
    for tweet in public_tweets:
        text = clean_text(tweet.full_text)
        data.append([tweet.created_at, tweet.user.screen_name, text])
df = pd.DataFrame(data, columns= column)
df.to_csv('Dataset/tweets.csv',mode='a', index=False, header=False)

In [12]:
df = pd.read_csv('C:/Users/SAMEER MISHRA/Desktop/COLLEGE PRACTICAL PROGRAM FILES/6th SEM/Project/Code/Dataset/tweets.csv', 
                 encoding = 'unicode_escape', 
                 engine ='python')
df.head(10)

,Time,User,Tweet
0,2022-04-25 12:17:47+00:00,NDRFHQ,cssr op stori const build collaps satya niketan new delhi ndrfghaziabad rescu live victim unconsci state debri cut fallen wall op continu hmoindia bhallaajay atulkarw pibhomeaffair
1,2022-04-14 03:12:30+00:00,NDRFHQ,honbl pm sh narendramodi virtual interact personnel involv ropeway rescu oper deoghar hail heroic effort coordin stakehold watch telecast committedserv
2,2022-04-13 18:04:03+00:00,NDRFHQ,committedserv aapdasevaalwayssarvatra
3,2022-04-11 07:57:45+00:00,NDRFHQ,rescu oper underway ropeway trikut devghar jh multi agenc coordin rescu person trolli op continu disasterservicealwayseverywher hmoindia bhallaajay ranchipib pibhomeaffair ndrfp
4,2022-04-10 04:30:55+00:00,NDRFHQ,whenev date written struggl written result braveri solemn valourday ceremoni crpfindia sh ajay kumar bhalla home secretari present gallantri medal sh atulkarw dg ndrf counter terror op srinagar
5,2022-04-09 17:57:27+00:00,NDRFHQ,humbl tribut brave crpf personnel made suprem sacrific battl sardar post gujarat pakistani brigad attack crpf valourday
6,2022-04-08 14:05:51+00:00,NDRFHQ,confer capac build disast respons discuss held humanitarian relief capac build sdrfcdhg fs role major stake holder emerg role ngos dm initi best practic sdrfs ani pibindia pibhomeaffair
7,2022-04-08 14:01:41+00:00,NDRFHQ,confer capac build disast respons conclud dr p k mishra princip secretari honbl pm grace valedictori session chief guest share view strengthen capac build pibhomeaffair ani
8,2022-04-08 04:11:22+00:00,NDRFHQ,annual confer capac build disast respons day apr
9,2022-04-07 15:52:36+00:00,NDRFHQ,confer capac build disast respons honbl home affair cooper minist sh amitshah presid inaugur session applaud ndrfs achiev stress need stakehold reach upto panchayat level pibhomeaffair ani
